In [1]:
# %matplotlib inline

import test
# import datetime as d
from datetime import datetime as dt
import matplotlib.pyplot as plt
import scipy.signal as sg
import pandas as pd
from sklearn.metrics import accuracy_score as acc
from sklearn.pipeline import make_pipeline as mp
from sklearn import model_selection as ms
import numpy as np

pd.plotting.register_matplotlib_converters()

## Get Data

In [2]:
Fs = 20
data_o = test.testbed(start=dt(2019,1,1),stop=dt(2021,8,8),Fs=Fs,interval='15minutes')


Starting dump run for date range 2019-01-01 00:00:00 - 2021-08-08 00:00:00 @interval=15minutes


Data acquired. Total data points:91169

Creating Parameter system of order:3...

Parameters are ready...

- Creating addtional features...

Data modelling complete...



C:\Users\mwill\AppData\Local\Programs\Python\Python37-32\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


- Performing Fourier Transform for v..

- Performing Fourier Transform for c..



- Dump Complete...


### Variables

In [3]:
columns = ['dv1t_o','dv2t_oo','dv3t_ooo','vf_t','dr1','ma1','x1wrk','x1te','x1ke','x1pe','ft1','w1','q1','x2gn','x1pr','x1wrk','idposc'
            ,'Fmm','Fma','Fmt','Mam','Maa','Mat','Qfm','Qfa','Qft','Pxm','Pxa','Pxt','Wkm','Wka','Wkt','Drm','Dra','Drt','vc_pwr','vc_c','x2wrk'
            ,'x2pr','x2gn','q2','w2','ft2','x2pe','x2ke','x2te','x2wrk','ma2','dr2','cf_t','dc3t_ooo','dc2t_oo','dc1t_o']

In [4]:
data_o.columns.values

array(['l', 'h', 'o', 'c', 'v', 'c1t_1', 'dc1t_o', 'c1rt_1', 'c1angt_1',
       'c2t_1', 'dc2t_o', 'c2rt_1', 'c2angt_1', 'c2to_1', 'dc2t_oo',
       'dc2angt_o', 'c3t_1', 'dc3t_o', 'c3rt_1', 'c3angt_1', 'c3to_1',
       'dc3t_oo', 'dc3angt_o', 'c3too_1', 'dc3t_ooo', 'v1t_1', 'dv1t_o',
       'v1rt_1', 'v1angt_1', 'v2t_1', 'dv2t_o', 'v2rt_1', 'v2angt_1',
       'v2to_1', 'dv2t_oo', 'dv2angt_o', 'v3t_1', 'dv3t_o', 'v3rt_1',
       'v3angt_1', 'v3to_1', 'dv3t_oo', 'dv3angt_o', 'v3too_1',
       'dv3t_ooo', 'vc_pwr', 'vc_c', 'idposc', 'al1', 'al2', 'x2gn',
       'x2gn', 'x1pr', 'x2pr', 'al1n', 'al2n', 'ft1', 'ft2', 'k1',
       'k2', 'ac1', 'ac2', 'ma1', 'ma2', 'w1n', 'w2n', 'dr1', 'dr2',
       'w1', 'w2', 'fr1', 'fr2', 'lmda1', 'lmda2', 'q1', 'q2', 'x1pe',
       'x1ke', 'x1te', 'x2pe', 'x2ke', 'x2te', 'x1wrk', 'x2wrk', 'Pxm', 'Pxa',
       'Pxt', 'Fmm', 'Fma', 'Fmt', 'Mam', 'Maa', 'Mat', 'Qfm', 'Qfa',
       'Qft', 'Drm', 'Dra', 'Drt', 'Wkm', 'Wka', 'Wkt', 'v_fft',
       'fft_freq', '

In [5]:
data_o[['w1n','w1','lmda1','dr1','al1','k1','k2','al2','dr2','lmda2','w2','w2n']].describe()

,w1n,w1,lmda1,dr1,al1,k1,k2,al2,dr2,lmda2,w2,w2n
count,9.115900e+04,91159.000000,9.115900e+04,91159.000000,91159.000000,9.115900e+04,9.115900e+04,91159.000000,91159.000000,91159.000000,91159.000000,9.115900e+04
mean,4.188744e+01,98.813197,-9.153315e+02,-21.852169,-0.983711,1.973849e+03,1.351808e+03,0.999421,18.372833,624.497667,0.049045,3.399028e+01
std,7.105466e-15,327.199154,1.138480e+05,2717.951406,122.353037,9.094997e-13,2.273749e-13,0.082111,1.509484,51.307766,3.109740,1.421093e-14
min,4.188744e+01,0.000000,-3.146557e+07,-751193.276903,-33816.196435,1.973849e+03,1.351808e+03,-8.521103,-156.647446,-5324.490079,0.000000,3.399028e+01
25%,4.188744e+01,0.000000,3.445105e+01,0.822467,0.037025,1.973849e+03,1.351808e+03,0.998397,18.353993,623.857292,0.000000,3.399028e+01
50%,4.188744e+01,0.000000,9.758649e+02,23.297313,1.048767,1.973849e+03,1.351808e+03,0.999986,18.383216,624.850606,0.000000,3.399028e+01
75%,4.188744e+01,17.611538,1.741466e+03,41.574900,1.871562,1.973849e+03,1.351808e+03,1.001426,18.409676,625.749980,0.000000,3.399028e+01
max,4.188744e+01,36304.459920,1.552098e+06,37054.020326,1668.047450,1.973849e+03,1.351808e+03,7.304141,134.275458,4564.060008,426.774216,3.399028e+01


In [70]:
sig = data_o[columns]

In [58]:
# Normalize training data
sign = sig.copy()

sign.reset_index(drop=True,inplace=True)

for col in sign:
        if col not in ['vf_t','cf_t','idposc']:
            sign[col] = (sign[col]-sign[col].mean())/sign[col].std()
            sign[col] = sign[col].replace([np.inf, -np.inf], np.nan)
            sign[col] = sign[col].fillna(np.abs(sign[col]).max())

In [69]:
sign.drop(['vf_t','cf_t','idposc'],axis=1).describe()

,dv1t_o,dv2t_oo,dv3t_ooo,dr1,ma1,x1wrk,x1te,x1ke,x1pe,ft1,...,ft2,x2pe,x2ke,x2te,x2wrk,ma2,dr2,dc3t_ooo,dc2t_oo,dc1t_o
count,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,...,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04
mean,1.273921e-18,-3.094677e-18,-3.139130e-18,2.689117e-18,6.485060e-17,-6.625361e-19,-1.629059e-17,-4.209053e-18,1.153592e-17,-2.574148e-17,...,1.169181e-18,4.520835e-18,-4.209053e-18,-1.714799e-17,-9.353451e-19,2.104527e-18,4.813520e-16,4.245590e-18,-3.059358e-18,-1.403018e-18
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.463199e+01,-3.559597e+01,-3.111087e+01,-2.763741e+02,-1.517755e+00,-1.064115e+02,-1.108596e+02,-1.033669e-01,-1.077174e-01,-2.836509e+01,...,-1.019073e+02,-1.075075e-01,-1.033669e-01,-1.149763e+02,-1.348588e+02,-9.867457e+00,-1.159471e+02,-3.281981e+01,-2.537749e+01,-3.183807e+01
25%,-1.939652e-01,-1.639722e-01,-2.052687e-01,8.342546e-03,-6.790584e-01,2.618491e-02,8.462437e-02,-1.027295e-01,-1.072408e-01,-1.474516e-01,...,-1.158322e-02,-1.070110e-01,-1.027295e-01,8.460421e-02,1.423494e-02,-5.088382e-01,-1.248104e-02,-8.208205e-02,-8.331922e-02,-7.728699e-02
50%,-9.019634e-03,3.250452e-02,2.640778e-03,1.661159e-02,-2.987253e-01,3.086535e-02,1.042673e-01,-9.905755e-02,-1.042739e-01,7.522433e-02,...,8.911052e-03,-1.040111e-01,-9.905755e-02,1.040034e-01,1.562043e-02,-1.856767e-03,6.878870e-03,1.342567e-05,-9.368347e-04,-3.030860e-03
75%,1.681193e-01,2.373379e-01,2.134236e-01,2.333635e-02,5.320597e-01,3.391893e-02,1.072335e-01,-7.759672e-02,-8.462727e-02,2.211074e-01,...,2.984584e-02,-8.460872e-02,-7.759672e-02,1.070014e-01,1.712261e-02,5.053892e-01,2.440787e-02,8.318911e-02,7.866861e-02,7.513349e-02
max,3.271898e+01,1.963168e+01,2.879750e+01,1.364111e+01,2.365784e+00,6.104047e+01,1.081428e-01,1.308705e+02,1.108554e+02,2.674257e+01,...,4.743823e+01,1.149547e+02,1.308705e+02,1.120779e-01,2.796973e+01,1.524986e+01,7.678296e+01,3.426623e+01,4.563335e+01,3.475014e+01


In [71]:
# Remove outliers
# for col in sign.columns:
#     q1 = sign[col].quantile(0.05)
#     q3 = sign[col].quantile(0.95)
#     iqr = q3-q1

#     sign_o = sign[((sign[col] < (q1-1.5*iqr)) | (sign[col] > (q3+1.5*iqr)))]
#     sign_no = sign[~((sign[col] < (q1-1.5*iqr)) | (sign[col] > (q3+1.5*iqr)))]


In [60]:
sign.dv1t_o.plot()
plt.title('signal v')
plt.show()

plt.magnitude_spectrum(sign.dv1t_o,Fs=Fs)
plt.show()

plt.phase_spectrum(sign.dv1t_o,Fs=Fs)
plt.show()

plt.angle_spectrum(sign.dv1t_o,Fs=Fs)
plt.show()

In [61]:
plt.specgram(sign.dv1t_o,Fs=Fs)
plt.title('specgram v')
plt.xlabel('dt')
plt.ylabel('fr')
plt.colorbar()

In [48]:
sign.dc1t_o.plot()
plt.title('signal c')
plt.show()

plt.magnitude_spectrum(sign.dc1t_o,Fs=Fs)
plt.show()

plt.phase_spectrum(sign.dc1t_o,Fs=Fs)
plt.show()

plt.angle_spectrum(sign.dc1t_o,Fs=Fs)
plt.show()

In [49]:
plt.specgram(sign.dc1t_o,Fs=Fs)
plt.title('specgram c')
plt.xlabel('dt')
plt.ylabel('fr')
plt.colorbar()

In [50]:
sign.vc_pwr.plot()
plt.title('signal power v*c')
plt.show()

plt.magnitude_spectrum(sign.vc_pwr,Fs=Fs)
plt.show()

plt.phase_spectrum(sign.vc_pwr,Fs=Fs)
plt.show()

plt.angle_spectrum(sig.vc_pwr,Fs=Fs)
plt.show()

In [81]:
sign.shape

(91159, 53)

In [101]:
plt.specgram(sign.vc_pwr,Fs=Fs)
plt.title('vc_pwr specgram')
plt.xlabel('dt')
plt.ylabel('fr')
plt.colorbar()

In [188]:
f,t,s = sg.spectrogram(sign.vc_pwr,fs=Fs)

import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(projection='3d')


ValueError: shape mismatch: objects cannot be broadcast to a single shape

## Strategies

In [7]:
idpos_alt1 = pd.Series(index=sig.index,data=np.where(sig.dc1t_o>0,1,0),name='idpos1')
idpos_alt2 = pd.Series(index=sig.index,data=np.where(sig.dc1t_o>(sig.dc1t_o.mean()+ 1.5*sig.dc1t_o.std()),1,0),name='idpos2')

sig_a1 = sig.join(idpos_alt1,on=sig.index)
sig_a2 = sig.join(idpos_alt2,on=sig.index)
sig_a1.describe()

C:\Users\mwill\AppData\Local\Programs\Python\Python37-32\lib\site-packages\numpy\core\_methods.py:48: ComplexWarning: Casting complex values to real discards the imaginary part
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,dv1t_o,dv2t_oo,dv3t_ooo,vf_t,dr1,ma1,x1wrk,x1te,x1ke,x1pe,...,x2ke,x2te,x2wrk,ma2,dr2,cf_t,dc3t_ooo,dc2t_oo,dc1t_o,idpos1
count,91159.000000,91159.000000,91159.000000,911590000.0+0.00000j,91159.000000,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,9.115900e+04,...,9.115900e+04,9.115900e+04,9.115900e+04,91159.000000,91159.000000,91159000.0+0.0000j,9.115900e+04,91159.00000,91159.000000,91159.000000
mean,0.043424,-0.011052,0.001048,0.0434240+0.0000000j,-21.852169,4.814462e+06,-1.074580e+10,-1.624371e+10,1.937363e+06,1.624565e+10,...,1.937363e+06,-4.034851e+09,-1.270886e+07,67.757026,18.372833,0.033265-0.000000j,-2.761963e-05,0.00006,0.033265,0.502956
std,2443.494297,1968.435677,776.520810,2443.494297+0.000000j,2717.951406,1.609831e+07,3.484335e+11,1.508103e+11,1.874258e+07,1.508174e+11,...,1.874258e+07,3.753537e+10,8.136786e+08,19276.837703,1.509484,7.676134+0.000000j,2.057225e+00,5.40164,7.676134,0.499994
min,-60188.087000,-70068.385500,-24158.239889,-60188.00087+0.00000j,-751193.276903,-1.961882e+07,-3.708807e+13,-1.673502e+13,0.000000e+00,0.000000e+00,...,0.000000e+00,-4.319713e+12,-1.097444e+11,-190145.604450,-156.647446,-244.30006+0.00000j,-6.751778e+01,-137.08000,-244.360000,0.000000
25%,-473.909500,-322.779750,-159.394333,-473.909005+0.000000j,0.822467,-6.117229e+06,-1.622105e+09,-3.481487e+09,1.194589e+04,7.187523e+07,...,1.194589e+04,-8.592007e+08,-1.126193e+06,-9741.034323,18.353993,-0.560000+0.000000j,-1.688889e-01,-0.45000,-0.560000,0.000000
50%,-21.996000,63.972000,2.051667,-21.9960000-0.0000000j,23.297313,5.490326e+03,8.719994e+06,-5.191288e+08,8.076835e+04,5.193402e+08,...,8.076835e+04,-1.310453e+08,1.153753e+03,31.964426,18.383216,0.010000-0.000000j,6.314393e-15,-0.00500,0.010000,1.000000
75%,410.842000,467.173250,165.728944,410.840002+0.000000j,41.574900,1.337972e+07,1.072688e+09,-7.180450e+07,4.829997e+05,3.482388e+09,...,4.829997e+05,-1.851513e+07,1.223449e+06,9810.061961,18.409676,0.610000-0.000000j,1.711111e-01,0.42500,0.610000,1.000000
max,79948.684000,38643.694000,22361.857556,79948.60084+0.00000j,37054.020326,4.289957e+07,2.125780e+13,6.533294e+07,2.454789e+09,1.673516e+13,...,2.454789e+09,1.720331e+08,2.274566e+10,294036.830456,134.275458,266.70008-0.00000j,7.049333e+01,246.49500,266.780000,1.000000


In [10]:
# Filter the dataset for position change pairs 0->1
sigb     = ((sig.idposc == 1) 
                        & (sig.idposc.shift(1) == 0)) | ((sig.idposc == 0) 

                        & (sig.idposc.shift(-1) == 1))

In [11]:
sigtr = sig[sigb]
sigtr.drop('idposc',axis=1).describe()

C:\Users\mwill\AppData\Local\Programs\Python\Python37-32\lib\site-packages\numpy\core\_methods.py:48: ComplexWarning: Casting complex values to real discards the imaginary part
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,dv1t_o,dv2t_oo,dv3t_ooo,vf_t,dr1,ma1,x1wrk,x1te,x1ke,x1pe,...,x2pe,x2ke,x2te,x2wrk,ma2,dr2,cf_t,dc3t_ooo,dc2t_oo,dc1t_o
count,6734.000000,6734.000000,6734.000000,6734.000000+0.000000j,6734.000000,6.734000e+03,6.734000e+03,6.734000e+03,6.734000e+03,6.734000e+03,...,6.734000e+03,6.734000e+03,6.734000e+03,6.734000e+03,6734.000000,6734.000000,6734.00000+0.00000j,6734.000000,6734.000000,6734.000000
mean,149.626892,32.270818,-1.104713,149.626892-0.000000j,1.897950,3.603956e+06,-1.343479e+10,-2.809646e+10,3.711589e+06,2.810017e+10,...,7.895145e+09,3.711589e+06,-7.891434e+09,-9.033187e+07,86.425363,18.549321,4.602631-0.000000j,1.493579,4.621602,4.602631
std,3409.040999,2724.518918,1059.998670,3409.040999+0.000000j,305.579456,1.477050e+07,3.790275e+11,1.587317e+11,2.039882e+07,1.587366e+11,...,3.393655e+10,2.039882e+07,3.392361e+10,1.794256e+09,18362.312407,2.264401,19.824623+0.000000j,5.189564,14.605547,19.824623
min,-26888.194000,-42107.513000,-15579.102222,-26888.10094-0.00000j,-10572.294780,-1.961262e+07,-1.354214e+13,-6.226019e+12,3.743570e-01,1.021408e+02,...,5.942823e+01,3.743570e-01,-1.110486e+12,-9.487444e+10,-119727.759597,-16.219148,-244.30006+0.00000j,-60.755556,-112.930000,-244.360000
25%,-905.441250,-614.449500,-295.085083,-905.441205+0.000000j,4.065732,-5.969833e+06,-5.576872e+09,-1.211827e+10,5.625569e+04,3.780020e+08,...,9.911470e+07,5.625569e+04,-3.522109e+09,-2.661131e+07,-8951.362285,18.324914,-4.082500-0.000000j,-0.627778,-1.876250,-4.082500
50%,18.561500,191.913500,19.451444,18.5615000+0.0000000j,24.113260,-2.258394e+06,1.102624e+08,-2.380408e+09,3.432228e+05,2.381446e+09,...,6.296090e+08,3.432228e+05,-6.291934e+08,2.614467e+06,242.980921,18.420911,7.700000+0.000000j,1.290556,4.390000,7.700000
75%,1002.441250,973.853625,331.998500,1002.441205+0.000000j,41.098361,1.256956e+07,5.877585e+09,-3.760634e+08,1.674918e+06,1.212139e+10,...,3.523313e+09,1.674918e+06,-9.897585e+07,4.279401e+07,9107.825801,18.584712,13.100000-0.000000j,3.390556,10.203750,13.100000
max,40522.033000,21438.434000,12110.607111,40522.00033+0.00000j,5023.238314,4.288835e+07,5.461837e+12,6.533294e+07,8.865215e+08,6.226094e+12,...,1.110704e+12,8.865215e+08,4.761934e+07,2.108074e+10,128430.064403,134.275458,266.70008-0.00000j,70.493333,246.495000,266.780000


## Test and Training Data

In [12]:
sig_ri = sig.reset_index(drop=True)
sig_ri.fillna(0,inplace=True)
xtrain,xtest,ytrain,ytest = ms.train_test_split(sig_ri.drop(['idposc'],axis=1),sig_ri.idposc,test_size=0.4)

In [13]:
# Normalize training data
for col in xtrain:
        if col != 'idposc':
            xtrain[col] = (xtrain[col]-xtrain[col].mean())/xtrain[col].std()
            xtrain[col] = xtrain[col]/np.abs(xtrain[col]).max()
            xtrain[col] = xtrain[col].replace([np.inf, -np.inf], np.nan)
            xtrain[col] = xtrain[col].fillna(np.abs(xtrain[col]).max())

C:\Users\mwill\AppData\Local\Programs\Python\Python37-32\lib\site-packages\numpy\core\_methods.py:48: ComplexWarning: Casting complex values to real discards the imaginary part
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


In [14]:
# Normalize testing data
for col in xtest.columns:
        if col != 'idposc':
            xtest[col] = (xtest[col]-xtest[col].mean())/xtest[col].std()
            xtest[col] = xtest[col]/np.abs(xtest[col]).max()
            xtest[col] = xtest[col].replace([np.inf, -np.inf], np.nan)
            xtest[col] = xtest[col].fillna(np.abs(xtest[col]).max())

In [15]:
xtrain.describe()

,dv1t_o,dv2t_oo,dv3t_ooo,vf_t,dr1,ma1,x1wrk,x1te,x1ke,x1pe,...,x2pe,x2ke,x2te,x2wrk,ma2,dr2,cf_t,dc3t_ooo,dc2t_oo,dc1t_o
count,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04+0.000000e+00j,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,...,5.469500e+04,5.469500e+04,5.469500e+04,54695.000000,5.469500e+04,5.469500e+04,5.469500e+04+0.000000e+00j,5.469500e+04,5.469500e+04,5.469500e+04
mean,-1.217906e-19,8.119375e-20,-5.683562e-20,-3.247750e-20-4.507158e-34j,3.552227e-21,6.454903e-18,-1.623875e-20,7.104453e-20,-2.841781e-20,-2.232828e-19,...,1.390443e-19,-2.841781e-20,-7.408930e-20,0.000000,6.657887e-19,-1.775606e-17,-1.380294e-19+3.028810e-34j,8.119375e-21,4.059687e-21,-1.014922e-19
std,3.703106e-02,2.847709e-02,3.209408e-02,3.703106e-02+0.000000e+00j,4.432694e-03,4.223477e-01,1.463568e-02,1.297379e-02,8.231705e-03,1.297430e-02,...,1.208039e-02,8.231705e-03,1.207699e-02,0.008316,6.605939e-02,9.388383e-03,2.909083e-02+0.000000e+00j,2.983063e-02,2.212434e-02,2.909083e-02
min,-9.010805e-01,-1.000000e+00,-1.000000e+00,-9.010805e-01+5.446032e-17j,-1.000000e+00,-6.435381e-01,-1.000000e+00,-1.000000e+00,-8.113964e-04,-1.577084e-03,...,-1.373399e-03,-8.113964e-04,-1.000000e+00,-1.000000,-6.470824e-01,-1.000000e+00,-9.162280e-01+1.066139e-16j,-9.576416e-01,-4.581107e-01,-9.162280e-01
25%,-7.049802e-03,-4.521407e-03,-6.521437e-03,-7.049802e-03-2.149399e-17j,3.047909e-05,-2.884368e-01,3.546755e-04,1.222258e-03,-8.065051e-04,-1.569863e-03,...,-1.367252e-03,-8.065051e-04,1.086948e-03,0.000124,-3.338681e-02,-1.003151e-04,-2.238894e-03-7.741079e-18j,-2.335812e-03,-1.803284e-03,-2.238894e-03
50%,-3.529578e-04,1.021466e-03,2.297517e-04,-3.529578e-04+9.389044e-18j,6.063514e-05,-1.221898e-01,4.341802e-04,1.524277e-03,-7.781784e-04,-1.524415e-03,...,-1.330271e-03,-7.781784e-04,1.329859e-03,0.000134,-3.556964e-04,6.681834e-05,-1.020042e-04+1.717328e-17j,7.558124e-05,1.985931e-06,-1.020042e-04
75%,6.098526e-03,6.804653e-03,7.074608e-03,6.098526e-03+9.568558e-18j,8.494550e-05,2.242302e-01,4.854745e-04,1.569705e-03,-6.120559e-04,-1.222380e-03,...,-1.087124e-03,-6.120559e-04,1.366811e-03,0.000145,3.333578e-02,2.200682e-04,2.147354e-03+3.782869e-18j,2.502735e-03,1.766688e-03,2.147354e-03
max,1.000000e+00,4.385937e-01,6.981746e-01,1.000000e+00+5.446032e-17j,4.935798e-02,1.000000e+00,7.197229e-01,1.580376e-03,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.430045e-03,0.192250,1.000000e+00,6.622370e-01,1.000000e+00-1.064880e-16j,1.000000e+00,1.000000e+00,1.000000e+00


In [16]:
xytrain = xtrain.join(ytrain, on=xtrain.index)
xytrain.describe()

C:\Users\mwill\AppData\Local\Programs\Python\Python37-32\lib\site-packages\numpy\core\_methods.py:48: ComplexWarning: Casting complex values to real discards the imaginary part
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,dv1t_o,dv2t_oo,dv3t_ooo,vf_t,dr1,ma1,x1wrk,x1te,x1ke,x1pe,...,x2ke,x2te,x2wrk,ma2,dr2,cf_t,dc3t_ooo,dc2t_oo,dc1t_o,idposc
count,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04+0.000000e+00j,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,...,5.469500e+04,5.469500e+04,54695.000000,5.469500e+04,5.469500e+04,5.469500e+04+0.000000e+00j,5.469500e+04,5.469500e+04,5.469500e+04,54695.000000
mean,-1.217906e-19,8.119375e-20,-5.683562e-20,-3.247750e-20-4.507158e-34j,3.552227e-21,6.454903e-18,-1.623875e-20,7.104453e-20,-2.841781e-20,-2.232828e-19,...,-2.841781e-20,-7.408930e-20,0.000000,6.657887e-19,-1.775606e-17,-1.380294e-19+3.028810e-34j,8.119375e-21,4.059687e-21,-1.014922e-19,0.050919
std,3.703106e-02,2.847709e-02,3.209408e-02,3.703106e-02+0.000000e+00j,4.432694e-03,4.223477e-01,1.463568e-02,1.297379e-02,8.231705e-03,1.297430e-02,...,8.231705e-03,1.207699e-02,0.008316,6.605939e-02,9.388383e-03,2.909083e-02+0.000000e+00j,2.983063e-02,2.212434e-02,2.909083e-02,0.219834
min,-9.010805e-01,-1.000000e+00,-1.000000e+00,-9.010805e-01+5.446032e-17j,-1.000000e+00,-6.435381e-01,-1.000000e+00,-1.000000e+00,-8.113964e-04,-1.577084e-03,...,-8.113964e-04,-1.000000e+00,-1.000000,-6.470824e-01,-1.000000e+00,-9.162280e-01+1.066139e-16j,-9.576416e-01,-4.581107e-01,-9.162280e-01,0.000000
25%,-7.049802e-03,-4.521407e-03,-6.521437e-03,-7.049802e-03-2.149399e-17j,3.047909e-05,-2.884368e-01,3.546755e-04,1.222258e-03,-8.065051e-04,-1.569863e-03,...,-8.065051e-04,1.086948e-03,0.000124,-3.338681e-02,-1.003151e-04,-2.238894e-03-7.741079e-18j,-2.335812e-03,-1.803284e-03,-2.238894e-03,0.000000
50%,-3.529578e-04,1.021466e-03,2.297517e-04,-3.529578e-04+9.389044e-18j,6.063514e-05,-1.221898e-01,4.341802e-04,1.524277e-03,-7.781784e-04,-1.524415e-03,...,-7.781784e-04,1.329859e-03,0.000134,-3.556964e-04,6.681834e-05,-1.020042e-04+1.717328e-17j,7.558124e-05,1.985931e-06,-1.020042e-04,0.000000
75%,6.098526e-03,6.804653e-03,7.074608e-03,6.098526e-03+9.568558e-18j,8.494550e-05,2.242302e-01,4.854745e-04,1.569705e-03,-6.120559e-04,-1.222380e-03,...,-6.120559e-04,1.366811e-03,0.000145,3.333578e-02,2.200682e-04,2.147354e-03+3.782869e-18j,2.502735e-03,1.766688e-03,2.147354e-03,0.000000
max,1.000000e+00,4.385937e-01,6.981746e-01,1.000000e+00+5.446032e-17j,4.935798e-02,1.000000e+00,7.197229e-01,1.580376e-03,1.000000e+00,1.000000e+00,...,1.000000e+00,1.430045e-03,0.192250,1.000000e+00,6.622370e-01,1.000000e+00-1.064880e-16j,1.000000e+00,1.000000e+00,1.000000e+00,1.000000


In [17]:
xytrain.corr()

C:\Users\mwill\AppData\Local\Programs\Python\Python37-32\lib\site-packages\pandas\core\internals\managers.py:1517: ComplexWarning: Casting complex values to real discards the imaginary part
  result[rl.indexer] = arr


,dv1t_o,dv2t_oo,dv3t_ooo,vf_t,dr1,ma1,x1wrk,x1te,x1ke,x1pe,...,x2ke,x2te,x2wrk,ma2,dr2,cf_t,dc3t_ooo,dc2t_oo,dc1t_o,idposc
dv1t_o,1.000000,0.812956,0.538562,1.000000,0.021925,-0.000615,0.252111,-0.002317,-0.286645,0.002272,...,-0.286645,-0.109840,0.013346,-0.001023,0.000814,-0.098690,0.008864,-0.038242,-0.098690,0.015743
dv2t_oo,0.812956,1.000000,0.890628,0.812956,0.028266,0.002569,-0.170119,-0.199904,-0.442136,0.199823,...,-0.442136,0.125430,0.058113,-0.000689,-0.003621,-0.061911,-0.031507,-0.065003,-0.061911,-0.003295
dv3t_ooo,0.538562,0.890628,1.000000,0.538562,0.028860,0.003922,-0.405277,-0.362760,-0.373834,0.362679,...,-0.373834,0.160471,0.057854,-0.001150,-0.005412,-0.030172,-0.052843,-0.058747,-0.030172,-0.005531
vf_t,1.000000,0.812956,0.538562,1.000000,0.021925,-0.000615,0.252111,-0.002317,-0.286645,0.002272,...,-0.286645,-0.109840,0.013346,-0.001023,0.000814,-0.098690,0.008864,-0.038242,-0.098690,0.015743
dr1,0.021925,0.028266,0.028860,0.021925,1.000000,0.002694,-0.001845,-0.000036,-0.005874,0.000035,...,-0.005874,0.003810,0.119647,-0.001787,-0.000190,-0.000491,-0.000337,-0.000598,-0.000491,0.001477
ma1,-0.000615,0.002569,0.003922,-0.000615,0.002694,1.000000,-0.003089,-0.002424,-0.000749,0.002423,...,-0.000749,0.003193,-0.000715,0.001417,0.001374,-0.002969,0.000583,-0.000085,-0.002969,-0.019782
x1wrk,0.252111,-0.170119,-0.405277,0.252111,-0.001845,-0.003089,1.000000,0.563990,0.249436,-0.563917,...,0.249436,-0.460739,-0.067817,0.002271,0.001826,-0.063251,0.034659,-0.000269,-0.063251,-0.009641
x1te,-0.002317,-0.199904,-0.362760,-0.002317,-0.000036,-0.002424,0.563990,1.000000,-0.378882,-1.000000,...,-0.378882,0.408163,0.033653,0.004379,0.001135,-0.006275,0.022858,0.004549,-0.006275,-0.034778
x1ke,-0.286645,-0.442136,-0.373834,-0.286645,-0.005874,-0.000749,0.249436,-0.378882,1.000000,0.379017,...,1.000000,-0.739027,-0.105562,-0.001583,0.000367,-0.002055,0.024527,0.031350,-0.002055,0.029696
x1pe,0.002272,0.199823,0.362679,0.002272,0.000035,0.002423,-0.563917,-1.000000,0.379017,1.000000,...,0.379017,-0.408255,-0.033668,-0.004379,-0.001135,0.006274,-0.022853,-0.004544,0.006274,0.034781


## Feature Selection

In [18]:
id_corr = xytrain.corr()['idposc']
id_corr.drop(['dc1t_o','idposc']).describe()

C:\Users\mwill\AppData\Local\Programs\Python\Python37-32\lib\site-packages\pandas\core\internals\managers.py:1517: ComplexWarning: Casting complex values to real discards the imaginary part
  result[rl.indexer] = arr


count    51.000000
mean      0.043486
std       0.114796
min      -0.040601
25%      -0.004576
50%       0.006312
75%       0.032239
max       0.539107
Name: idposc, dtype: float64

In [19]:
sp = id_corr.describe()['75%'] - id_corr.describe()['50%']
sp

0.03019132264118146

In [20]:
t = abs(id_corr) > sp
t = list(t.index[t==True])
id_corr[t].drop('idposc').sort_values(ascending=False)

dc1t_o      0.539107
cf_t        0.539107
x2pr       0.460074
dc2t_oo     0.378812
dc3t_ooo    0.192756
Pxt         0.120623
Pxm         0.120296
Fmm         0.106205
Fmt         0.105960
ft2         0.065718
x2pe         0.040601
Wkt         0.038181
Wkm         0.038180
x1pe         0.034781
x1te        -0.034778
x2te        -0.040601
Name: idposc, dtype: float64

In [21]:
xtest.describe()

C:\Users\mwill\AppData\Local\Programs\Python\Python37-32\lib\site-packages\numpy\core\_methods.py:48: ComplexWarning: Casting complex values to real discards the imaginary part
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,dv1t_o,dv2t_oo,dv3t_ooo,vf_t,dr1,ma1,x1wrk,x1te,x1ke,x1pe,...,x2pe,x2ke,x2te,x2wrk,ma2,dr2,cf_t,dc3t_ooo,dc2t_oo,dc1t_o
count,3.646400e+04,3.646400e+04,36464.000000,3.646400e+04+0.000000e+00j,3.646400e+04,3.646400e+04,3.646400e+04,3.646400e+04,3.646400e+04,3.646400e+04,...,3.646400e+04,3.646400e+04,3.646400e+04,3.646400e+04,3.646400e+04,3.646400e+04,3.646400e+04+0.000000e+00j,3.646400e+04,3.646400e+04,3.646400e+04
mean,8.525188e-20,-8.525188e-20,0.000000,1.035201e-19-3.434392e-34j,2.131297e-20,-8.379042e-18,-3.044710e-21,-1.050425e-19,6.089420e-20,1.004754e-19,...,-1.826826e-19,6.089420e-20,3.653652e-20,-1.522355e-20,6.820150e-19,-6.312597e-17,-6.698362e-20-2.379736e-34j,3.958123e-20,7.307304e-20,-4.871536e-20
std,2.999389e-02,3.169368e-02,0.034822,2.999389e-02+0.000000e+00j,6.931898e-03,4.232323e-01,1.101561e-02,1.073512e-02,8.841845e-03,1.073542e-02,...,9.089205e-03,8.841845e-03,9.088076e-03,7.388773e-03,9.432696e-02,2.212897e-02,2.978954e-02+0.000000e+00j,3.282130e-02,2.769487e-02,2.978954e-02
min,-6.858302e-01,-1.000000e+00,-0.882664,-6.858302e-01-4.550400e-17j,-1.000000e+00,-6.385624e-01,-1.000000e+00,-1.000000e+00,-1.005991e-03,-1.031185e-03,...,-9.006349e-04,-1.005991e-03,-1.000000e+00,-1.000000e+00,-7.168181e-01,-1.000000e+00,-8.927627e-01-1.429305e-20j,-7.419545e-01,-7.132901e-01,-8.927627e-01
25%,-5.994870e-03,-5.434336e-03,-0.007220,-5.994870e-03-4.664792e-18j,1.140098e-04,-2.848968e-01,3.191789e-04,8.254225e-04,-9.995601e-04,-1.026839e-03,...,-8.962931e-04,-9.995601e-04,7.022853e-04,9.908699e-05,-4.848798e-02,-3.589750e-04,-2.317761e-03-1.061647e-18j,-2.902879e-03,-2.385043e-03,-2.317761e-03
50%,-2.468023e-04,8.682139e-04,-0.000131,-2.468023e-04+1.334123e-17j,2.278138e-04,-1.280535e-01,3.628392e-04,1.000219e-03,-9.629153e-04,-1.000309e-03,...,-8.698167e-04,-9.629153e-04,8.696078e-04,1.120376e-04,2.833114e-04,1.448619e-04,-6.846536e-05-1.206742e-17j,-1.309074e-04,-6.964537e-05,-6.846536e-05
75%,5.186410e-03,7.443766e-03,0.007068,5.186410e-03-9.508615e-18j,3.215751e-04,2.258398e-01,3.917636e-04,1.026745e-03,-7.480932e-04,-8.253542e-04,...,-7.024059e-04,-7.480932e-04,8.960413e-04,1.265078e-04,4.790323e-02,5.923621e-04,2.259753e-03+2.596590e-17j,2.696136e-03,2.141689e-03,2.259753e-03
max,1.000000e+00,6.351122e-01,1.000000,1.000000e+00+1.277041e-21j,1.206163e-01,1.000000e+00,5.737412e-01,1.034990e-03,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,9.115131e-04,2.638491e-01,1.000000e+00,9.525619e-01,1.000000e+00+5.606363e-17j,1.000000e+00,1.000000e+00,1.000000e+00


In [22]:
xtrain_sel = xytrain[t].drop(['idposc','dc1t_o'],axis=1)
xtrain_sel.describe()

C:\Users\mwill\AppData\Local\Programs\Python\Python37-32\lib\site-packages\numpy\core\_methods.py:48: ComplexWarning: Casting complex values to real discards the imaginary part
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,x1te,x1pe,Fmm,Fmt,Pxm,Pxt,Wkm,Wkt,x2pr,ft2,x2pe,x2te,cf_t,dc3t_ooo,dc2t_oo
count,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04,5.469500e+04+0.000000e+00j,5.469500e+04,5.469500e+04
mean,7.104453e-20,-2.232828e-19,-1.299100e-19,-6.739081e-19,1.997366e-18,-4.059687e-21,5.683562e-20,-2.151634e-19,4.343866e-19,-6.089531e-21,1.390443e-19,-7.408930e-20,-1.380294e-19+3.028810e-34j,8.119375e-21,4.059687e-21
std,1.297379e-02,1.297430e-02,4.055534e-02,4.055620e-02,3.370249e-02,3.369294e-02,1.454641e-02,1.454343e-02,3.387637e-02,9.736167e-03,1.208039e-02,1.207699e-02,2.909083e-02+0.000000e+00j,2.983063e-02,2.212434e-02
min,-1.000000e+00,-1.577084e-03,-2.112626e-02,-5.197119e-02,-1.678983e-02,-1.726818e-02,-1.843858e-03,-6.214156e-03,-9.134853e-01,-1.000000e+00,-1.373399e-03,-1.000000e+00,-9.162280e-01+1.066139e-16j,-9.576416e-01,-4.581107e-01
25%,1.222258e-03,-1.569863e-03,-1.794095e-02,-1.794406e-02,-1.427249e-02,-1.427121e-02,-1.835023e-03,-1.834426e-03,-1.228034e-02,-1.106624e-04,-1.367252e-03,1.086948e-03,-2.238894e-03-7.741079e-18j,-2.335812e-03,-1.803284e-03
50%,1.524277e-03,-1.524415e-03,-1.248356e-02,-1.247756e-02,-1.011424e-02,-1.011583e-02,-1.780685e-03,-1.780275e-03,-1.196959e-05,9.050875e-05,-1.330271e-03,1.329859e-03,-1.020042e-04+1.717328e-17j,7.558124e-05,1.985931e-06
75%,1.569705e-03,-1.222380e-03,1.158020e-03,1.156462e-03,3.984687e-04,4.042379e-04,-1.420920e-03,-1.420737e-03,1.239952e-02,2.956462e-04,-1.087124e-03,1.366811e-03,2.147354e-03+3.782869e-18j,2.502735e-03,1.766688e-03
max,1.580376e-03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.655088e-01,1.000000e+00,1.430045e-03,1.000000e+00-1.064880e-16j,1.000000e+00,1.000000e+00


False

In [24]:
np.all(np.isfinite(xtrain_sel))
np.any(np.isnan(xtrain_sel))

True

In [25]:
xtrain_sel.convert_dtypes(convert_integer=False,convert_boolean=False,convert_floating=True,infer_objects=True)

TypeError: complex128 cannot be converted to a FloatingDtype

In [ ]:
def warn(*args,**kwargs):
    pass

## Models

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score as cvs
from sklearn.linear_model import LogisticRegressionCV
import warnings

warnings.warn=warn

adb = AdaBoostClassifier()
svm = SVC(gamma='scale',kernel='rbf')
gbc = GradientBoostingClassifier()
rfc = RandomForestClassifier()
dtc = DecisionTreeClassifier()
knn = KNeighborsClassifier()

logr = LogisticRegressionCV(solver='lbfgs')


### Model Cross Validation

In [ ]:
seed = 42
models = []
results = []
names = []
matrix = []

models.append(('ADA', AdaBoostClassifier()))
models.append(('DTC',DecisionTreeClassifier()))
models.append(('GB', GradientBoostingClassifier()))
models.append(('KNN',KNeighborsClassifier()))
models.append(('LR', LogisticRegressionCV(solver='lbfgs',max_iter=5000,cv=10)))
models.append(('RF',RandomForestClassifier()))
models.append(('SVC',SVC(gamma='scale',kernel='rbf')))

In [ ]:
for name, model in models:
    cv_results = cvs(model,xtrain_sel,ytrain,scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    accuracy = [name,cv_results.mean(),cv_results.std()]
    matrix.append(accuracy)

In [ ]:
matrix

### Gradient Boost

In [ ]:
model = gbc.fit(xtrain_sel,ytrain)
ypred = model.predict(xtest[xtrain_sel.columns])
accuracy = acc(ytest,ypred)

In [ ]:
accuracy

In [ ]:
model.get_params()

In [ ]:
df_pred = pd.DataFrame(index=xtest.index,data=ypred,columns=['predpos'])

test = xtest[xtrain_sel.columns].join(ytest).join(df_pred)

test.describe()

### Random Forest

In [ ]:
model = rfc.fit(xtrain_sel,ytrain)

ypred = model.predict(xtest[xtrain_sel.columns])
accuracy = acc(ytest,ypred)

In [ ]:
accuracy

In [ ]:
model.get_params()

In [ ]:
df_pred = pd.DataFrame(index=xtest.index,data=ypred,columns=['predpos'])

test = xtest[xtrain_sel.columns].join(ytest).join(df_pred)

test.describe()

### ADA Boost

In [ ]:
model = adb.fit(xtrain_sel,ytrain)

ypred = model.predict(xtest[xtrain_sel.columns])
accuracy = acc(ytest,ypred)

In [ ]:
accuracy

In [ ]:
model.get_params()

In [ ]:
df_pred = pd.DataFrame(index=xtest.index,data=ypred,columns=['predpos'])

test = xtest[xtrain_sel.columns].join(ytest).join(df_pred)

test.describe()

### K Nearest Neighbor

In [ ]:
model = knn.fit(xtrain_sel,ytrain)
ypred = model.predict(xtest[xtrain_sel.columns])
accuracy = acc(ytest,ypred)

In [ ]:
accuracy

In [ ]:
model.get_params()

In [ ]:
df_pred = pd.DataFrame(index=xtest.index,data=ypred,columns=['predpos'])

test = xtest[xtrain_sel.columns].join(ytest).join(df_pred)

test.describe()

### Logistic Regression

In [ ]:
model = logr.fit(xtrain_sel,ytrain)
ypred = model.predict(xtest[xtrain_sel.columns])
accuracy = acc(ytest,ypred)

In [ ]:
accuracy

In [ ]:
model.get_params()

In [ ]:
df_pred = pd.DataFrame(index=xtest.index,data=ypred,columns=['predpos'])

test = xtest[xtrain_sel.columns].join(ytest).join(df_pred)

test.describe()

### Decision Tree

In [ ]:
model = dtc.fit(xtrain_sel,ytrain)
ypred = model.predict(xtest[xtrain_sel.columns])
accuracy = acc(ytest,ypred)

In [ ]:
accuracy

In [ ]:
model.get_params()

In [ ]:
df_pred = pd.DataFrame(index=xtest.index,data=ypred,columns=['predpos'])

test = xtest[xtrain_sel.columns].join(ytest).join(df_pred)

test.describe()

### Support Vector Machine

In [ ]:
model = svm.fit(xtrain_sel,ytrain)
ypred = model.predict(xtest[xtrain_sel.columns])
accuracy = acc(ytest,ypred)

In [ ]:
accuracy

In [ ]:
model.get_params()

In [ ]:
df_pred = pd.DataFrame(index=xtest.index,data=ypred,columns=['predpos'])

test = xtest[xtrain_sel.columns].join(ytest).join(df_pred)

test.describe()

In [ ]:
sig.plot.scatter('dv1t_o','dc1t_o')